In [ ]:
import warnings
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.decomposition import NMF
np.random.seed(100)
warnings.filterwarnings("ignore")

In [ ]:
def generate_action_set(M, d):
    
    complete_rating = pd.read_csv('complete_ratings').values[:, 1:] / 5
 
    model = NMF(n_components = int(np.sqrt(d)), init = 'nndsvda', max_iter = 10000)
    W = model.fit_transform(complete_rating)
    H = model.components_
 
    theta_true = np.eye(int(np.sqrt(d))).ravel()
    Theta = np.tile(theta_true, (M, 1)).T
    Theta[:, M - 2] = np.ones(d) / np.sqrt(d)
    Theta[:, M - 1] = np.ones(d) / np.sqrt(d)
 
    kmeans = KMeans(n_clusters = M, random_state = 0).fit(H.T)
    labels = kmeans.labels_
 
    Action_list = np.empty((W.shape[0], M), dtype = object)
 
    for t in range(W.shape[0]):
 
        for i in range(M):

            columns_cluster = np.where(labels == i)[0]
            size = len(columns_cluster)
            Action_list[t, i] = np.empty((size, d), dtype = float)
 
            for ac, column_idx in enumerate(columns_cluster):
                
                outer = np.outer(W[t], H[:, column_idx])

                if i == M - 2 or i == M - 1:

                    diag_values = W[t] * H[:, column_idx]
                    Action_list[t, i][ac] = np.tile(diag_values[:, None], (1, int(np.sqrt(d)))).ravel()

                else:

                    Action_list[t, i][ac] = outer.ravel
                    
    return Action_list, Theta